In [1]:
import numpy as np

import torch
import torch.nn as nn

# from torch_geometric.nn import GCNConv

In [2]:
class GCNConvByHand(nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=True)

    def forward(self, x, A):
        num_nodes = A.shape[0]
        omega_k = self.linear.weight
        beta_k = self.linear.bias.reshape(-1, 1)
        # print("omega_k", omega_k, "beta_k", beta_k)
        H_k = x
        x = torch.matmul(
            beta_k, torch.reshape(torch.ones(num_nodes), (1, -1))
        ) + torch.matmul(omega_k, torch.matmul(H_k, A + torch.eye(num_nodes)))
        return x

In [3]:
class GCNByHand(nn.Module):
    def __init__(self, num_nodes, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConvByHand(in_channels, in_channels)
        self.conv2 = GCNConvByHand(in_channels, in_channels)
        self.out = torch.nn.Linear(num_nodes, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        print("x after first layer\n", x)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        print("x after second layer\n", x)
        x = torch.relu(x)
        x = self.out(x)
        print("x after output layer\n", x)
        x = torch.sigmoid(x)
        return x

In [4]:
num_nodes = 3
num_features = 2

x = torch.randn((num_features, num_nodes))
# print("x", x, x.shape)

edge_index = torch.tensor([[0, 0, 1, 2], [1, 2, 0, 0]], dtype=torch.int64)

A = torch.tensor([[0, 1, 1], [1, 0, 0], [1, 0, 0]])  # adjacency matrix

model = GCNByHand(num_nodes=num_nodes, in_channels=num_features, out_channels=num_nodes)

out = model(x, A)

print("output")
print(out)

x after first layer
 tensor([[ 0.2669,  0.3220,  0.2914],
        [-0.5228,  0.2084, -0.1128]], grad_fn=<AddBackward0>)
x after second layer
 tensor([[-0.4921, -0.4260, -0.4223],
        [ 0.7491,  0.6256,  0.6861]], grad_fn=<AddBackward0>)
x after output layer
 tensor([[-0.0534,  0.5001,  0.5242],
        [ 0.3470,  1.0205,  0.0261]], grad_fn=<AddmmBackward0>)
output
tensor([[0.4867, 0.6225, 0.6281],
        [0.5859, 0.7351, 0.5065]], grad_fn=<SigmoidBackward0>)


In [5]:
A = np.array(A)

H_k = np.array(x)
# H_k

# H_k__A = H_k @ A
# H_k__A, H_k__A.shape

omega_0 = np.array(model.conv1.linear.weight.detach().numpy())
beta_0 = np.array(model.conv1.linear.bias.detach().numpy()).reshape((-1, 1))
# print(omega_0, beta_0)


preactivation = beta_0 @ np.ones(num_nodes).reshape((1, -1)) + omega_0 @ H_k @ (
    A + np.identity(num_nodes)
)
preactivation

array([[ 0.26693794,  0.32202418,  0.29138244],
       [-0.52278422,  0.20843466, -0.11282943]])